In [2]:
import pandas as pd
import geopandas as gpd
import arcpy
import os
import shutil
import datetime as dt
import math
THIS_FOLDER = os.path.dirname(os.path.abspath("RPP-SBC-EEMS"))
print(THIS_FOLDER)

C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS


In [3]:
arcpy.ResetEnvironments()
gdb_name = "Reporting_Units.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)
if arcpy.Exists(gdb_name):
    print("Already Exists")
else:
    try:
        arcpy.CreateFileGDB_management("./", gdb_name)
    except:
        print("{} already exists".format(gdb_name))
        pass
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path

Already Exists


In [4]:
data = ["in_data/Pr_Lg_Fire.shp", "in_data/Pr_Fire_Ign.shp",
        "in_data/Pct_Cg_Lg_Fi.shp", "in_data/Pct_Cg_Fi_Ign.shp",
        "in_data/WUI_Intermix.shp", "in_data/WUI_Interface.shp",
        "in_data/FloraFaunaMerge.shp",
        "out_data/lu/LU_merged.shp", "out_data/ru/Reporting_Units.shp",
        "out_data/building/Building_Intersect.shp",
        "in_data/Slope.shp", "in_data/Veg_Only.shp",
        "in_data/FHSZ.shp"]

for v in data:
    path = os.path.join(THIS_FOLDER, "{}".format(v))
    name = v.rsplit('/', 1)[-1].split(".")[0]
    arcpy.CopyFeatures_management(path, name)
    crs = arcpy.SpatialReference('NAD 1983 StatePlane California V FIPS 0405 (US Feet)')
    arcpy.Project_management(name, "{}_rpj".format(name), crs)

In [5]:
arcpy.analysis.Intersect("Reporting_Units_rpj; Pr_Lg_Fire_rpj", "A_RU_0_int", "ALL", None, "INPUT")

arcpy.SummarizeWithin_analysis("Reporting_Units_rpj", "A_RU_0_int", 
                               "A_RU_1", "KEEP_ALL", [['gridcode', 'MEAN']],
                               "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Reporting_Units.gdb\A_RU_1
1,


In [6]:
arcpy.analysis.Intersect("Pr_Fire_Ign_rpj; A_RU_1", "A_RU_1_int", "ALL", None, "INPUT")

arcpy.SummarizeWithin_analysis("A_RU_1", "A_RU_1_int", 
                               "A_RU_2", "KEEP_ALL", [['gridcode', 'MEAN']],
                               "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Reporting_Units.gdb\A_RU_2
1,


In [7]:
arcpy.analysis.Intersect("Pct_Cg_Lg_Fi_rpj; A_RU_2", "A_RU_2_int", "ALL", None, "INPUT")

arcpy.SummarizeWithin_analysis("A_RU_2", "A_RU_2_int", 
                               "A_RU_3", "KEEP_ALL", [['gridcode', 'MEAN']],
                               "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Reporting_Units.gdb\A_RU_3
1,


In [8]:
arcpy.analysis.Intersect("A_RU_3; Pct_Cg_Fi_Ign_rpj", "A_RU_3_int", "ALL", None, "INPUT")

arcpy.SummarizeWithin_analysis("A_RU_3", "A_RU_3_int", 
                               "A_RU_4", "KEEP_ALL", [['gridcode', 'MEAN']],
                               "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Reporting_Units.gdb\A_RU_4
1,


In [9]:
arcpy.analysis.Intersect("A_RU_4; WUI_Intermix_rpj", "A_RU_4_int", "ALL", None, "INPUT")

arcpy.SummarizeWithin_analysis("A_RU_4", "A_RU_4_int", 
                               "A_RU_5", "KEEP_ALL", [['SHAPE_AREA', 'MEAN']],
                               "ADD_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Reporting_Units.gdb\A_RU_5
1,


In [10]:
arcpy.analysis.Intersect("A_RU_5; WUI_Interface_rpj", "A_RU_5_int", "ALL", None, "INPUT")

arcpy.SummarizeWithin_analysis("A_RU_5", "A_RU_5_int", 
                               "A_RU_6", "KEEP_ALL", [['SHAPE_AREA', 'MEAN']],
                               "ADD_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Reporting_Units.gdb\A_RU_6
1,


In [11]:
arcpy.analysis.Intersect("A_RU_6; LU_Merged_rpj", "A_RU_6_int", "ALL", None, "INPUT")

arcpy.SummarizeWithin_analysis("A_RU_6", "A_RU_6_int", 
                               "A_RU_7", "KEEP_ALL", [['LU_Cat', 'Max']],
                               "NO_SHAPE_SUM", "SQUAREFEET", "LU_Cat", "ADD_MIN_MAJ", "ADD_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Reporting_Units.gdb\A_RU_7
1,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Reporting_Units.gdb\LU_Cat_Summary


In [12]:
arcpy.analysis.Intersect("A_RU_7; FloraFaunaMerge_rpj", "A_RU_7_int", "ALL", None, "INPUT")

arcpy.SummarizeWithin_analysis("A_RU_7", "A_RU_7_int", 
                               "A_RU_8", "KEEP_ALL", [['Flora_Faun', 'Mean']],
                               "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Reporting_Units.gdb\A_RU_8
1,


In [13]:
arcpy.analysis.Intersect("A_RU_8; Building_Intersect_rpj", "A_RU_8_int", "ALL", None, "INPUT")

arcpy.analysis.SummarizeWithin("A_RU_8", "A_RU_8_int", 
                               "A_RU_9", "KEEP_ALL", "", 
                               "ADD_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Reporting_Units.gdb\A_RU_9
1,


In [14]:
arcpy.analysis.Intersect("A_RU_9; Slope_rpj", "A_RU_9_int", "ALL", None, "INPUT")

arcpy.analysis.SummarizeWithin("A_RU_9", "A_RU_9_int", 
                               "A_RU_10", "KEEP_ALL", "gridcode Mean", 
                               "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Reporting_Units.gdb\A_RU_10
1,


In [15]:
arcpy.analysis.Intersect("A_RU_10; Veg_Only_rpj", "A_RU_10_int", "ALL", None, "INPUT")

arcpy.analysis.SummarizeWithin("A_RU_10", "A_RU_10_int", 
                               "A_RU_11", "KEEP_ALL", "gridcode Max", 
                               "NO_SHAPE_SUM", "SQUAREFEET", "gridcode", "ADD_MIN_MAJ", "ADD_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Reporting_Units.gdb\A_RU_11
1,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Reporting_Units.gdb\gridcode_Summary


In [34]:
arcpy.analysis.Intersect("A_RU_11; FHSZ_rpj", "A_RU_11_int", "ALL", None, "INPUT")

arcpy.analysis.SummarizeWithin("A_RU_11", "A_RU_11_int", 
                               "A_RU_12", "KEEP_ALL", "HAZ_CODE Max", 
                               "NO_SHAPE_SUM", "SQUAREFEET", "HAZ_CODE", "ADD_MIN_MAJ", "ADD_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Reporting_Units.gdb\A_RU_12
1,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Reporting_Units.gdb\HAZ_CODE_Summary


In [35]:
arcpy.AddField_management("A_RU_12", "WUI_Interm", "LONG")
arcpy.AddField_management("A_RU_12", "WUI_Interf", "LONG")

codeblock = """
def get_val(m):
    if m is None:
        return 0
    elif m > 0:
        return 1
    else:
        return 0
"""

# Execute CalculateField 
arcpy.CalculateField_management("A_RU_12", "WUI_Interm", "get_val(!mean_shape_area!)", "PYTHON3", codeblock)
arcpy.CalculateField_management("A_RU_12", "WUI_Interf", "get_val(!mean_shape_area_1!)", "PYTHON3", codeblock)

arcpy.DeleteField_management("A_RU_12", 
                             ['SUM_Area_SQUAREFEET', 'SUM_Area_SQUAREFEET_1'])

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Reporting_Units.gdb\\A_RU_12'>

In [36]:
arcpy.AddField_management("A_RU_12", "Mj_LU_Cat", "LONG")

codeblock = """
def get_val(a, m):
    if m is None:
        return 0
    elif m >= 50:
        return a
    else:
        return 0
"""

# Execute CalculateField 
arcpy.CalculateField_management("A_RU_12", "Mj_LU_Cat", "get_val(!Majority_LU_Cat!, !Majority_LU_Cat_Percent!)", "PYTHON3", codeblock)

arcpy.DeleteField_management("A_RU_12", 
                             ['MAX_LU_Cat', 'Minority_LU_Cat', 'Majority_LU_Cat', 'Majority_LU_Cat_Percent', 'Minority_LU_Cat_Percent'])

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Reporting_Units.gdb\\A_RU_12'>

In [37]:
arcpy.AddField_management("A_RU_12", "Mj_Haz_Cat", "LONG")

codeblock = """
def get_val(a, m):
    if m is None:
        return 0
    elif m >= 50:
        return a
    else:
        return 0
"""

# Execute CalculateField 
arcpy.CalculateField_management("A_RU_12", "Mj_Haz_Cat", "get_val(!Majority_HAZ_CODE!, !Majority_HAZ_CODE_Percent!)", "PYTHON3", codeblock)

arcpy.DeleteField_management("A_RU_12", 
                             ['MAX_HAZ_CODE', 'Minority_HAZ_CODE', 'Majority_HAZ_CODE', 'Majority_HAZ_CODE_Percent', 'Minority_HAZ_CODE_Percent'])

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Reporting_Units.gdb\\A_RU_12'>

In [38]:
arcpy.AddField_management("A_RU_12", "Vg_Pct", "FLOAT")

codeblock = """
def get_val(a, m):
    if m is None:
        return 0
    else:
        return round(m/100, 2)
"""

# Execute CalculateField 
arcpy.CalculateField_management("A_RU_12", "Vg_Pct", "get_val(!Majority_gridcode!, !Majority_gridcode_Percent!)", "PYTHON3", codeblock)

arcpy.DeleteField_management("A_RU_12", 
                             ['MAX_gridcode', 'Minority_gridcode', 'Minority_gridcode_Percent'])

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Reporting_Units.gdb\\A_RU_12'>

In [39]:
fieldObs = arcpy.ListFields("A_RU_12")  

fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)
del fieldObs  
fieldCount = len(fieldNames) 

with arcpy.da.UpdateCursor("A_RU_12", fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0"  
        curU.updateRow(rowU)

del curU

print(fieldNames)

['OBJECTID', 'Shape', 'Shape_Leng', 'mean_gridcode', 'Polygon_Count', 'mean_gridcode_1', 'Polygon_Count_1', 'mean_gridcode_12', 'Polygon_Count_12', 'mean_gridcode_12_13', 'Polygon_Count_12_13', 'mean_shape_area', 'Polygon_Count_12_13_14', 'mean_shape_area_1', 'Polygon_Count_12_13_14_15', 'sum_Area_SQUAREFEET_12', 'Polygon_Count_12_13_14_15_16', 'Join_ID', 'mean_flora_faun', 'Polygon_Count_12_13_14_15_16_17', 'sum_Area_SQUAREFEET_12_13', 'Polygon_Count_12_13_14_15_16_17_18', 'mean_gridcode_12_13_14', 'Polygon_Count_12_13_14_15_16_17_18_19', 'sum_Area_SQUAREFEET_12_13_14', 'Polygon_Count_12_13_14_15_16_17_18_19_20', 'Majority_gridcode', 'Majority_gridcode_Percent', 'Join_ID_1', 'Shape_Length', 'Shape_Area', 'sum_Area_SQUAREFEET_12_13_14_15', 'Polygon_Count_12_13_14_15_16_17_18_19_20_21', 'Join_ID_1_2', 'WUI_Interm', 'WUI_Interf', 'Mj_LU_Cat', 'Mj_Haz_Cat', 'Vg_Pct']


In [40]:
arcpy.AlterField_management('A_RU_12', 
                            'MEAN_gridcode', 'Pr_Lg_Fr', 'Pr_Lg_Fr')
arcpy.AlterField_management('A_RU_12', 
                            'MEAN_gridcode_1', 'Pr_Fi_Ign', 'Pr_Fi_Ign')
arcpy.AlterField_management('A_RU_12', 
                            'MEAN_gridcode_12', 'Pc_Cg_LgFi', 'Pc_Cg_LgFi')
arcpy.AlterField_management('A_RU_12', 
                            'MEAN_gridcode_12_13', 'Pc_Fi_Ign', 'Pc_Fi_Ign')
arcpy.AlterField_management('A_RU_12', 
                            'MEAN_gridcode_12_13_14', 'Slp', 'Slp')

arcpy.AlterField_management('A_RU_12', 
                            'MEAN_Flora_Faun', 'Fl_Fa', 'Fl_Fa')

arcpy.AlterField_management('A_RU_12', 
                            'SUM_Area_SQUAREFEET_12_13', 'Buil_Ar', 'Buil_Ar')

arcpy.DeleteField_management("A_RU_12", 
                             ['SUM_Area_SQUAREFEET_12', 'Join_ID_1', 'Shape_Leng', 'SUM_Area_SQUAREFEET_12_13_14', 
                              'SUM_Area_SQUAREFEET_12_13_14_15', "Majority_gridcode", "Majority_gridcode_Percent"])

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Reporting_Units.gdb\\A_RU_12'>

In [41]:
arcpy.AddField_management("A_RU_12", "Buil_Rat", "FLOAT")

arcpy.CalculateField_management("A_RU_12", "Buil_Rat", 
                                '!Buil_Ar!/!Shape_Area!', "PYTHON3")

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Reporting_Units.gdb\\A_RU_12'>

In [42]:
fieldObs = arcpy.ListFields("A_RU_12") 

fieldNames = []  
for field in fieldObs: 
    print(field.name)
    fieldNames.append(field.name)  
del fieldObs

OBJECTID
Shape
Pr_Lg_Fr
Polygon_Count
Pr_Fi_Ign
Polygon_Count_1
Pc_Cg_LgFi
Polygon_Count_12
Pc_Fi_Ign
Polygon_Count_12_13
mean_shape_area
Polygon_Count_12_13_14
mean_shape_area_1
Polygon_Count_12_13_14_15
Polygon_Count_12_13_14_15_16
Join_ID
Fl_Fa
Polygon_Count_12_13_14_15_16_17
Buil_Ar
Polygon_Count_12_13_14_15_16_17_18
Slp
Polygon_Count_12_13_14_15_16_17_18_19
Polygon_Count_12_13_14_15_16_17_18_19_20
Shape_Length
Shape_Area
Polygon_Count_12_13_14_15_16_17_18_19_20_21
Join_ID_1_2
WUI_Interm
WUI_Interf
Mj_LU_Cat
Mj_Haz_Cat
Vg_Pct
Buil_Rat


In [43]:
dir_path = os.path.join(THIS_FOLDER, "out_data/eems")
shutil.rmtree(dir_path)
os.mkdir(dir_path)

arcpy.conversion.FeatureClassToShapefile("A_RU_12", 
                                        os.path.join(THIS_FOLDER, "out_data/eems"))

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\out_data\\eems'>

In [44]:
gdf = gpd.read_file(os.path.join(THIS_FOLDER, "out_data/eems/A_RU_12.shp"))
gdf = gdf[['Join_ID', 'Pr_Lg_Fr', 'Pr_Fi_Ign', 'Pc_Cg_LgFi', 'Pc_Fi_Ign',
           'Fl_Fa', 'Buil_Ar', 'Slp', 'Shape_Leng', 'Shape_Area', 'WUI_Interm',
           'WUI_Interf', 'Mj_LU_Cat', 'Mj_Haz_Cat', 'Vg_Pct', 'Buil_Rat', 'geometry']]
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 18176 entries, 0 to 18175
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Join_ID     18176 non-null  int64   
 1   Pr_Lg_Fr    18176 non-null  float64 
 2   Pr_Fi_Ign   18176 non-null  float64 
 3   Pc_Cg_LgFi  18176 non-null  float64 
 4   Pc_Fi_Ign   18176 non-null  float64 
 5   Fl_Fa       18176 non-null  float64 
 6   Buil_Ar     18176 non-null  float64 
 7   Slp         18176 non-null  float64 
 8   Shape_Leng  18176 non-null  float64 
 9   Shape_Area  18176 non-null  float64 
 10  WUI_Interm  18176 non-null  int64   
 11  WUI_Interf  18176 non-null  int64   
 12  Mj_LU_Cat   18176 non-null  int64   
 13  Mj_Haz_Cat  18176 non-null  int64   
 14  Vg_Pct      18176 non-null  float64 
 15  Buil_Rat    18176 non-null  float64 
 16  geometry    18176 non-null  geometry
dtypes: float64(11), geometry(1), int64(5)
memory usage: 2.4 MB


In [45]:
gdf['geometry'] = gdf['geometry'].to_crs(epsg=26911)

In [46]:
gdf.to_csv("out_data/eems/Reporting_Units.csv", index=False)